# **Predict User New Word**

<img src='https://cdn.analyticsvidhya.com/wp-content/uploads/2023/07/Screenshot_from_2023-07-14_11-04-52.png' width=500>

In [ ]:
#  this import library

from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping


In [ ]:
#  import my dataset

with open('/content/1661-0.txt') as f:
  data = f.readlines()


In [ ]:
# check head and tail

print('head:', data[:5])
print('tail:', data[-5:])


head: ['\ufeff\n', "Project Gutenberg's The Adventures of Sherlock Holmes, by Arthur Conan Doyle\n", '\n', 'This eBook is for the use of anyone anywhere at no cost and with\n', 'almost no restrictions whatsoever.  You may copy it, give it away or\n']
tail: ['\n', '\n', '\n', '\n', '\n']


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense



# Tokenization
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(data)
total_words = len(tokenizer.word_index) + 1

# Create input sequences
input_sequences = []
for line in data:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Pad sequences
max_sequence_length = max([len(seq) for seq in input_sequences])
input_sequences = tf.keras.preprocessing.sequence.pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre')

# Create predictors and labels
X, y = input_sequences[:, :-1], input_sequences[:, -1]
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

# Build the model
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_length-1))
model.add(LSTM(100))
model.add(Dense(total_words, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=100, verbose=1)


Epoch 1/100
3176/3176 [==============================] - 153s 47ms/step - loss: 6.3289 - accuracy: 0.0707
Epoch 2/100
3176/3176 [==============================] - 150s 47ms/step - loss: 5.5641 - accuracy: 0.1232
Epoch 3/100
3176/3176 [==============================] - 151s 48ms/step - loss: 5.1689 - accuracy: 0.1495
Epoch 4/100
3176/3176 [==============================] - 155s 49ms/step - loss: 4.8639 - accuracy: 0.1676
Epoch 5/100
3176/3176 [==============================] - 155s 49ms/step - loss: 4.5955 - accuracy: 0.1835
Epoch 6/100
3176/3176 [==============================] - 154s 48ms/step - loss: 4.3488 - accuracy: 0.2002
Epoch 7/100
3176/3176 [==============================] - 154s 48ms/step - loss: 4.1157 - accuracy: 0.2202
Epoch 8/100
3176/3176 [==============================] - 152s 48ms/step - loss: 3.8951 - accuracy: 0.2409
Epoch 9/100
3176/3176 [==============================] - 150s 47ms/step - loss: 3.6840 - accuracy: 0.2660
Epoch 10/100
3176/3176 [======================

# prediction

In [ ]:
# de  for predict new sentence   of user

def predict_next_word(model, tokenizer, text):
  """
  This function predicts the next word in a given sentence.

  Args:
    model: The trained model.
    tokenizer: The tokenizer used to convert text to sequences.
    text: The sentence for which the next word is to be predicted.

  Returns:
    The predicted next word.
  """

  sequence = tokenizer.texts_to_sequences([text])
  sequence = tf.keras.preprocessing.sequence.pad_sequences(sequence, maxlen=max_sequence_length-1, padding='pre')
  preds = np.argmax(model.predict(sequence))
  predicted_word = ""

  for key, value in tokenizer.word_index.items():
      if value == preds:
          predicted_word = key
          break

  return predicted_word

# Get user input sentence
input_sentence = input("Enter your sentence: ")

# Predict the next word
next_word = predict_next_word(model, tokenizer, input_sentence)

# Print the predicted next word
print("Predicted next word:", next_word)


Enter your sentence:  i am 
1/1 [==============================] - 0s 25ms/step
Predicted next word: far


# Try to predict new word of user

In [ ]:
# predict new words list  of user

def predict_new_words(seed_text, model, tokenizer, max_sequence_length, num_words):
    """
    Predicts a list of new words based on a seed text.

    Args:
        seed_text: The seed text to start the prediction.
        model: The trained LSTM model.
        tokenizer: The tokenizer used to convert text to sequences.
        max_sequence_length: The maximum length of the input sequences.
        num_words: The number of new words to predict.

    Returns:
        A list of predicted words.
    """

    # Initialize an empty list to store the predicted words.
    predicted_words = []

    # Start with the seed text.
    current_text = seed_text

    # Loop for the specified number of words.



    for _ in range(num_words):

        # Tokenize the current text.
        token_list = tokenizer.texts_to_sequences([current_text])[0]

        # Pad the sequence.
        token_list = tf.keras.preprocessing.sequence.pad_sequences([token_list], maxlen=max_sequence_length-1, padding='pre')

        # Predict the next word probabilities.
        predicted_probs = model.predict(token_list, verbose=0)[0]

        # Get the index of the word with maximum probability.
        predicted_index = np.argmax(predicted_probs)

        # Map the index to the corresponding word.
        predicted_word = tokenizer.index_word.get(predicted_index, "")

        # Add the predicted word to the list.
        predicted_words.append(predicted_word)

        # Update the current text with the predicted word.
        current_text += " " + predicted_word

    # Return the list of predicted words.
    return predicted_words


# Example usage
seed_text = "i am "
predicted_words = predict_new_words(seed_text, model, tokenizer, max_sequence_length, 5)
print(f"Predicted words: {predicted_words}")


Predicted words: ['far', 'from', 'to', 'remember', 'that']
